In [1]:
# https://github.com/onnx/tensorflow-onnx/blob/master/tutorials/huggingface-bert.ipynb

# Converting a Huggingface model to ONNX with tf2onnx

This is a simple example how to convert a [huggingface](https://huggingface.co/) model to ONNX using [tf2onnx](https://github.com/onnx/tensorflow-onnx).

We use the [TFBertForQuestionAnswering](https://huggingface.co/transformers/model_doc/bert.html#tfbertforquestionanswering) example from huggingface.

Other models will work similar. You'll find additional examples for other models in our unit tests [here](https://github.com/onnx/tensorflow-onnx/blob/master/tests/huggingface.py).

## Install dependencies

In [2]:
!pip install tensorflow transformers tf2onnx onnxruntime

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 435 kB 69.5 MB/s 
     |████████████████████████████████| 4.8 MB 23.5 MB/s 
     |████████████████████████████████| 3.3 MB 18.3 MB/s 
     |████████████████████████████████| 61 kB 497 kB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
     |████████████████████████████████| 895 kB 45.3 MB/s 
     |████████████████████████████████| 12.7 MB 22.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


## The keras code

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import onnxruntime as rt
import tensorflow as tf
import tf2onnx
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

In [4]:
! wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt


--2021-12-06 08:02:05--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.86.149
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.86.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘bert-base-uncased-vocab.txt’

bert-base-uncased-v 100%[===================>] 226.08K  --.-KB/s    in 0.1s    

2021-12-06 08:02:06 (1.90 MB/s) - ‘bert-base-uncased-vocab.txt’ saved [231508/231508]



In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

test_sentence = "With their homes in ashes, residents share "

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
text = """
Voted Boston Globe\s 2018 best New England beach town! Narragansett is the perfect seaside community for a laid back coastal retreat. Immaculate, highly ranked by VRBO, 4+ bedroom three and a half bath Pottery Barn inspired shingle style beach home designed for families and entertaining. Main part of the house has four bedrooms two and a half baths with an outdoor shower and changing room. Additionally, there is a large, finished lower level GUEST SUITE with a kitchenette, fourth bathroom and queen sized bed in the sleeping area. Perfect for family getaways, reunions, anniversaries, graduations. and local wedding accommodations. Comfortable for as little as 4 to as many as 12.\n \n\nThe Harbour Island community is set on a peninsula that jets out into the salt pond. Enjoy barbecuing on our expanded deck leading onto one of the largest back yards on the island, or simply walk to kayaking, clamming, and paddle boarding on Point Judith.\n\nOur property is surrounded by four of Narragansett‚Äôs finest beaches just minutes away, and just up the street form a quaint private association beach/swim area (dock is for members only). Close to restaurants, shopping, Block Island ferry, and the center of town. Available primarily in the summer months weekly, monthly, or seasonally with advanced notice. \n\n*** HOUSE AMENITIES ***\n\nTwo Story Lofted Great Room, Central AC, WiFi, Custom Kitchen, Stainless Steel Appliances, Stunning Guest Suite With Private Bath and Kitchenette, Outdoor Dining Area, Large Yard, Gas Fire Pit With Seating, Weber Grill, Quality Pots and Pans, Keurig and Conventional Coffee, Mosquito and Tick Controlled, Stone Patio, Hammock, Front Porch, Multiple TV\s, Outdoor Shower/Changing Area, Town Beach Passes, Walk to Private Quaint Association Beach (Dock is For Members Only), Off Street Parking For Up To Six Cars, Walk To Paddle Boarding, Kayaking, Clamming and Sunsets\n\nImmaculate, 3,000 sq/ft Pottery Barn inspired beach house... Featured in the style section of "SO RI" magazine! Centrally located on Harbour Island. Four bedrooms with additional lower level Guest Suite for accommodating even more guests. Open floor plan for entertaining, custom kitchen, huge yard for grilling and relaxing, central AC, deck, loft, outdoor shower, stone patio, close to beaches, kayaking,paddle boarding, B. I. ferry, shopping, & restaurants.\n\nAbout the area...\nHarbour Island is centrally located in Narragansett. Our private Association Beach and swim area is just down the street from our front door (dock is for members only). It\s a great place to take a dip, catch some sun, or launch a kayak or paddle board. Narragansett Town beach passes provided...Sand Hill Cove, Galilee, Scarborough Beach, and the Block Island ferry are all within 2.5 to 4 miles of the property. Harbour Island is close to shops, restaurants, center of town, and market yet you feel like your in a quaint coastal setting. Newport, Providence, and Foxwoods casino are all short drives away.\nAbout the home...We have one of the largest yards on the island for grilling and relaxing with a new expanded patio and fire pit off our main deck so there is plenty of room for the whole gang! Home was built by owner to entertain with a wide open floor plan, lofted great room w/ 20 foot ceilings, top of the line kitchen, central air, porch, deck, custom built-ins, fire pit and distant water views from upper level. There is an emphasis on open, airy, light filled rooms.\nMain part of the home has four bedrooms two and one half baths, and a hot and cold outdoor shower with changing room. Additionally, there is a large family room with fifth queen bed, fourth bath, microwave and fridge in the lower level to accommodate even more guests. Comfortable for as little as 4 to as many as 12. We have many returning guests year after...We do our best to make your stay as comfortable as possible providing incidentals such as stainless cookware, blankets, comforters, pillows, beach towels, bath towels, lawn chairs, lawn games, trash bags, spare propane for Weber grill, laundry/dish detergent, etc. Fresh sheets and pillow cases arrive at your front door from Vacation Beach House Linens the day of your arrival. Smaller, hypoallergenic pets will be considered only after speaking with host.	
"""


In [29]:
%%timeit
predict_input = tokenizer.encode(text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = model.predict(predict_input)[0]
# print("tf_output:  ", tf_output)
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
# print("tf_prediction: ",  tf_prediction)

1 loop, best of 5: 2.03 s per loop


## Convert to ONNX

In [21]:
# describe the inputs
input_spec = (
    tf.TensorSpec((None,  None), tf.int32, name="input_ids"),
    # tf.TensorSpec((None,  None), tf.int32, name="token_type_ids"),
    # tf.TensorSpec((None,  None), tf.int32, name="attention_mask")
)

# and convert
_, _ = tf2onnx.convert.from_keras(model, input_signature=input_spec, opset=13, output_path="bert.onnx")

In [22]:
!du -hs /content/bert.onnx

419M	/content/bert.onnx


## Test the ONNX model with onnxruntime

In [23]:
opt = rt.SessionOptions()
ort_session = rt.InferenceSession("bert.onnx")

In [32]:
%%timeit
tokenized_text = tokenizer.encode(text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
input_ids=tokenized_text
input_dict = {"input_ids" : input_ids.numpy() }#, "attention_mask" : attention_masks.numpy() , "token_type_ids": all_tokens.numpy() }
onnx_results = ort_session.run(None, input_dict)
# onnx_results

1 loop, best of 5: 1.72 s per loop


## Make sure tensorflow and onnxruntime results are the same

In [31]:
# outputs are matching
np.testing.assert_allclose(tf_results['logits'], onnx_results[0], rtol=1e-5, atol=1e-5)

NameError: ignored